## 自定义输出解析器

In [3]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

question = "LangChain是什么？"

### 纳美星语解析器
纳美星语是人造语言，有2000个词汇
输入自然语言转为纳美星语

In [ ]:
import re
from langchain_core.messages import AIMessage, AIMessageChunk


# 定义一个函数作为自定义解析器
def parse(ai_message: AIMessage) -> str:
    """Convert message to Na'vi style language."""
    content = ai_message.content

    # 纳美星语特征转换规则
    navi_rules = {
        # 常见问候语转换
        r"\b[Hh]ello\b": "Kaltxì",
        r"\b[Hh]i\b": "Kaltxì",
        r"\b[Tt]hank you\b": "Irayo",
        r"\b[Gg]oodbye\b": "Ngaru",
        # 提娜姬纳美语特征的后缀
        r"\b(friend|friends)\b": "eylan",
        r"\b(brother|brothers)\b": "tsmukan",
        r"\b(sister|sisters)\b": "tsmuke",
        # 添加纳美语语气词
        r"[!.]\s*$": "kxe!",
        r"\?\s*$": "srak?",
        # 在某些词前添加特征前缀
        r"\b(beautiful|prettuy)\b": "na'vi",
        r"\b(sacred|holy)\b": "txe'lan",
    }

    # 应用转换规则
    result = content
    for pattern, replacement in navi_rules.items():
        result = re.sub(pattern, replacement, result)

    # 添加纳美语特征词缀
    result = f"Oel ngati kameie ... {result}"

    return result


chain = llm | parse
chain.invoke("hello")

'Oel ngati kameie ... Kaltxì! 😊 How can I assist you todaysrak?'

### 布尔值解析器
输入自然语言，通过自定义解析器变为true或false

In [ ]:
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.exceptions import OutputParserException

# 通过集成BaseOutputParser实现更复杂的输出解析器


# [bool] 描述一个泛型的参数化
# 它基本上说明了解析的返回类型是什么
# 在这种情况下，返回类型为True或False
# 自定义解析器将YES、NO字符串解析为布尔值
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""

    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"BooleanOutputParser expected output value to either be"
                f"{self.true_val} or {self.false_val} (case-insensitive)."
                f"Received {cleaned_text}."
            )
        return cleaned_text == self.true_val.upper()

    # 设置统一接口名
    @property
    def _type(self) -> str:
        return "boolean_output_parser"

    def get_format_instructions(self) -> str:
        return "Output should be 'YES' or 'NO'."


parser = BooleanOutputParser()
# 如果传入值不是YES 或 NO会报错
try:
    # parser.invoke("MEOW")
    parser.invoke("yes")
except OutputParserException as e:
    print(f"Triggered an exception of type: {type(e)}")


YES
NO


In [26]:
# 也可以重写参数
parser = BooleanOutputParser(true_val="OKAY", false_val="NOKY")
parser.invoke("OKAY")


OKAY
NOKY


True